# Needed imports and utility

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from comtypes.client import CreateObject

import ansys.speos.core as core
from ansys.speos.core.speos import Speos
import ansys.speos.workflow as workflow

tests_data_path defines the asset files location, where pre-prepared files are used in this demonstration.
In this simple demonstration, we have prepared one triangle prism geometry and flat surface source.
Of course, you can also use or create your own geometry from mesh information.

In [ ]:
tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

Some utility functions are defined at the beginning:
function print_message: Print rpc simulation progress message.
function open_file: Open and display the simulation result.

In [ ]:
def print_message(msg):
    print(core.protobuf_message_to_str(msg))

def open_file(file):
    dpf_instance = None
    if file.endswith("xmp") or file.endswith("XMP"):
        dpf_instance = CreateObject("XMPViewer.Application")
        dpf_instance.OpenFile(file)
        res = dpf_instance.ExportXMPImage(file+".png", 1)
        if res:
            img = mpimg.imread(file+".png")
            plt.imshow(img)
            plt.axis("off")   # turns off axes
            plt.axis("tight")  # gets rid of white border
            plt.axis("image")  # square up the image instead of filling the "figure" space
            plt.show()
    elif file.endswith("hdr") or file.endswith("HDR"):
        dpf_instance = CreateObject("HDRIViewer.Application")
        dpf_instance.OpenFile(file)
        dpf_instance.Show(1)
    elif file.endswith("png") or file.endswith("PNG"):
        img = mpimg.imread(file)
        plt.imshow(img)
        plt.axis("off")   # turns off axes
        plt.axis("tight")  # gets rid of white border
        plt.axis("image")  # square up the image instead of filling the "figure" space
        plt.show()
    return dpf_instance

# Create connection with speos rpc server
In this simple demonstration, a local host server is used. However, such server can be a remote server or can be on cloud.

Firstly, a speos client is initiated.
This speos client will later provide all the necessary rpc API for us to create workflow.

After a speos client is created, it is always best practice to clean all the database inside.
clean_all_dbs function from workflow class is defined to clean all the database in the memory.

In [ ]:
speos = Speos(host="localhost", port=50051)
workflow.clean_all_dbs(speos.client)

# Create SpeosSimulationUpdate
After a client is created and cleaned, it is empty, i.e. there is nothing inside.
Thus, we need add different objects.
Just like how we see the world, there are
* sources: sun, lamp, etc
* objects: table, food, street, and buildings, etc
* detectors: our eye or cameras
SpeosSimulationupdate is an object which is initialized to host all the necessary information required to run a simulation.
Firstly, an empty SpeosSimulationUpdate, named ssu, is initialized.

In [ ]:
ssu = workflow.SpeosSimulationUpdate(speos=speos, clean_dbs=False)
print(ssu.status)
print(ssu.scene)

Then, we want to add a geometry, i.e. triangle-prism in this case.
sim_file is file path where a triangle-prism geometry is pre-prepared with glass material linked. Of course, you can also create your own geometry.
an SpeosSimulationupdate object, named prism_scene, for triangle-prism is initialized with a given speos client and optionally file path.
Instead of using it for simulation, it will be added into ssu.
when adding, it is necessary to define where in the space you want to add it. Thus, you can specify the 3d space location.
a position information is created defined by origin and x, y, z axis vectors.
the unit used is mm by default. You can use preview function to check how different origin coordinate and x, y, z vectors change its positions.
using add_scene method, prism_scene is added into ssu.

In [ ]:
prism_file = os.path.join(tests_data_path, "Prism.speos", "Prism.speos")
prism_scene = workflow.SpeosSimulationUpdate(speos=speos, file_name=prism_file, clean_dbs=False)
prism_position = core.AxisSystem()
prism_position.origin = [0.0, 0.0, 0.0]
prism_position.x_vect = [1.0, 0.0, 0.0]
prism_position.y_vect = [0.0, 1.0, 0.0]
prism_position.z_vect = [0.0, 0.0, 1.0]

ssu.add_scene(prism_scene, prism_position)

# Preview geometries inside the simulation
preview function allows you to have a 3D view about the geometries added.

In [ ]:
ssu.preview()

# Add source into the simulation and preview source and geometry locations
The, to be able to "see" something, light source needs to be added.
In this demonstration, a pre-define surface shape source is added to the simulation:
the position is also pre-defined by the global coordinate of prim, thus in this example, we just need to put at global origin.

In [ ]:
source_file = os.path.join(tests_data_path, "Source.speos", "Source.speos")
source_scene = workflow.SpeosSimulationUpdate(speos=speos, file_name=source_file, clean_dbs=False)
source_position = core.AxisSystem()
ssu.add_scene(source_scene, source_position)

ssu.preview()

# New Irradiance sensor
A sensor is like a detector to receive result/light at defined position & orientation & target information to be recorded.
Here, a plane-type sensor is chosen to detect the refraction light from the triangle prism.
Default parameter are used in the first sensor

In [ ]:
irradiance_sensor = workflow.IrradianceSensorParameters()

irradiance_props = workflow.IrradianceSensorProperties()
irradiance_props.origin = [921.36, 0, 388.72]
irradiance_props.x_vect = [0.388721970152395, 0, -0.921355105223193]
irradiance_props.y_vect = [0.0, 1.0, 0.0]
irradiance_props.z_vect = [0.921355105223193, 0.0, 0.388721970152395]

ssu.add_irradiance_sensor(irradiance_sensor, irradiance_props)

More parameters for the second sensor are defined.
There are some interesting parameters that you can play, e.g.
type: this gives option for you to study photometric result, i.e. photometric energy received by detector. Or colormetric result if you are interested in certain wavelength visual result, or spectral which gives extra information at different wavelength value.

In [ ]:
irradiance_sensor_new = irradiance_sensor.copy() # assign values to parameters
irradiance_sensor_new.name = "new_sensor"
irradiance_sensor_new.integration_type = core.SensorTemplateFactory.IlluminanceType.Planar
irradiance_sensor_new.type = core.SensorTemplateFactory.Type.Spectral
irradiance_sensor_new.wavelengths_start = 380
irradiance_sensor_new.wavelengths_end = 780
irradiance_sensor_new.wavelengths_sampling = 41
irradiance_sensor_new.x_range_start = -40
irradiance_sensor_new.x_range_end = 40
irradiance_sensor_new.x_range_sampling = 400
irradiance_sensor_new.y_range_start = -5
irradiance_sensor_new.y_range_end = 5
irradiance_sensor_new.y_range_sampling = 100

ssu.add_irradiance_sensor(irradiance_sensor_new, irradiance_props)

# Compute simulation
With all the necessary info added, i.e. source, geometry, and detector.
we can run the simulation.
By default, compute run the simulation on CPU.
Optionally, compute can run on GPU with parameter compute_type set as "GPU"/"gpu".

In [ ]:
job_link = ssu.compute("first_job")
print_message(job_link.get_results())

# Open and review results

In [ ]:
open_file(job_link.get_results().results[2].path)
open_file(job_link.get_results().results[1].path)

# Modify the irradiance sensor
The detector information can be modified using update_sensor method.
e.g. change detector settings like wavelength range for infrared range
e.g. change detector positions and orientation

In [ ]:
irradiance_sensor_update = irradiance_sensor_new.copy()

irradiance_props_update = workflow.IrradianceSensorProperties()
irradiance_props_update.origin = [1000, 0, 400]
irradiance_props_update.x_vect = [0.388721970152395, 0, -0.921355105223193]
irradiance_props_update.y_vect = [0.0, 1.0, 0.0]
irradiance_props_update.z_vect = [0.921355105223193, 0.0, 0.388721970152395]

ssu.update_sensor(irradiance_sensor_update, irradiance_props_update)

# Compute the modified simulation

In [ ]:
job_link = ssu.compute("second_job")
print_message(job_link.get_results())

# Open and view the modified result

In [ ]:
open_file(job_link.get_results().results[2].path)

# Compute the modified simulation using GPU
Note: a student license does not support GPU compute

In [ ]:
job_link = ssu.compute("second_job", compute_type="gpu")
print_message(job_link.get_results())

# Open the view the GPU simulation result

In [ ]:
open_file(job_link.get_results().results[2].path)

# Clean after usage

In [ ]:
job_link.delete()
ssu.close()
print(ssu.status)